# Create Raster Template

In [1]:
# Import libraries

# System

import os

# Analysis

import numpy as np # Numerical Analysis
import geopandas as gpd # Spatial Data Mgmt
import rasterio # Rasters
from rasterio.transform import Affine # Transformations

In [2]:
# Definitions

def save_geotiff(array, name, crs, resolution, minx, miny):
    '''Saves a numpy array into a geotiff.
    
    Give name as a string
    crs as int, resolution as int
    minx and miny both as floats
    '''
    transform = Affine.translation(minx - resolution / 2, miny - resolution / 2\
                              ) * Affine.scale(resolution, resolution)
    
    with rasterio.open(
        os.path.join(".", name + '.tif'),
        mode="w",
        driver="GTiff",
        height=array.shape[1],
        width=array.shape[0],
        count=1,
        dtype= 'float64',
        crs= rasterio.crs.CRS.from_epsg(crs),
        transform=transform,
    ) as new_dataset:
        new_dataset.write(array, 1)

In [3]:
# Load Data

# Minneapolis Boundary

mpls_path = os.path.join(os.getcwd(), '..', '1_Data_IO', 'Data', 'mpls_boundary.geojson')
mpls = gpd.read_file(mpls_path)

## Prep

### Define Boundary

In [4]:
# Let's start with just stuff within 8 km of minneapolis

mpls_buffed = mpls.geometry.buffer(8000)

# Get important info

extent = mpls_buffed.geometry.total_bounds # minx, miny, maxx, maxy

minx = extent[0]
miny = extent[1]
maxx = extent[2]
maxy = extent[3]

height = extent[3] - extent[1]
width = extent[2] - extent[0]

print('height = ', height, 'width = ', width)

height =  33855.989475003444 width =  26629.013039288344


### Create Raster

In [5]:
# Create raster

resolution = 50 # Want maybe 50 meter x 50 meter cells

# Find the bounding box corrections 
#   needed to make a raster with this resolution 

x_correction = resolution - np.mod(width, resolution)
y_correction = resolution - np.mod(height, resolution)

# Split correction amongst min & maxs

new_minx = minx - x_correction/2
new_maxx = maxx + x_correction/2
new_miny = miny - y_correction/2
new_maxy = maxy + y_correction/2

# New heights

height = new_maxy - new_miny
width = new_maxx - new_minx

print('Corrected Height (for 50m resolution) = ', height, 'width = ', width)

x_cells = width/resolution
y_cells = height/resolution

print('x_cells = ', x_cells, '\ny_cells = ', y_cells)

Corrected Height (for 50m resolution) =  33900.0 width =  26650.0
x_cells =  533.0 
y_cells =  678.0


In [6]:
# Create Numpy Grid - Must mannual input cells from above.......

raster = np.mgrid[new_minx:new_maxx:533j,
              new_miny:new_maxy:678j]

## Save

In [7]:
# Save Template as numpy raster

np.save('template', raster)

In [8]:
# Save Template as a GeoTiff

array_temp = np.empty([int(x_cells), int(y_cells)])

save_geotiff(array_temp, 'template', 26915, resolution, minx, miny)